Aprendizaje Automático - Práctica 1
Autores: RICARDO ANTONIO PAZOS VALERO - 100472303 / VICENTE ANTONIO BARBATO - 10043114

# 6. Problema de clasificación

### Primera Parte: Usar modelo final para comprobar predicciones de valores altos y valores bajos

In [1]:
# Importamos
from joblib import load
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Cargamos datos
data = pd.read_csv('data/wind_ava.csv')

# Cargamos el modelo final
final_model = load('modelo_final.pkl')

# Filtramos los datos para ver solo los datos metereológicos obtenidos para Sotavento 
data_filter = [columna for columna in data.columns if columna.endswith('.13') or columna == "energy"]
data_sotavento = data[data_filter]
data_sotavento = data_sotavento.copy()  # Hacemos una copia de los datos para evitar el SettingWithCopyWarning

# Definimos el tercer cuantil
energy_tq = data_sotavento['energy'].quantile(0.75)

# Creamos una nueva variable target - la definimos como una variable binaria (más fácil de trabajar con variables binarias)
# Clase alta = 1
# Clase baja = 0
data_sotavento['energy_level'] = (data_sotavento['energy'] > energy_tq).astype(int)

# Hacemos las predicciones con el modelo final
predictions = final_model.predict(data_sotavento.drop(columns=['energy', 'energy_level']))

# Separamos los datos en dos subsets de energía alta o energía baja
high_energy = data_sotavento[data_sotavento['energy_level'] == 1]
low_energy = data_sotavento[data_sotavento['energy_level'] == 0]

# Hacemos predicciones para cada subset por separado
high_energy_predictions = predictions[high_energy.index]
low_energy_predictions = predictions[low_energy.index]

# Evaluamos el modelo en cada subset por separado
high_energy_mae = mean_absolute_error(high_energy['energy'], high_energy_predictions)
high_energy_mse = mean_squared_error(high_energy['energy'], high_energy_predictions)
high_energy_r2 = r2_score(high_energy['energy'], high_energy_predictions)

low_energy_mae = mean_absolute_error(low_energy['energy'], low_energy_predictions)
low_energy_mse = mean_squared_error(low_energy['energy'], low_energy_predictions)
low_energy_r2 = r2_score(low_energy['energy'], low_energy_predictions)

# Comparamos métricas
print("Métricas - Energía alta:")
print("MAE:", high_energy_mae)
print("MSE:", high_energy_mse)
print("R^2:", high_energy_r2)

print("\nMétricas - Energía baja:")
print("MAE:", low_energy_mae)
print("MSE:", low_energy_mse)
print("R^2:", low_energy_r2)

Métricas - Energía alta:
MAE: 363.7577378572341
MSE: 239320.74546830868
R^2: -0.4971161919850622

Métricas - Energía baja:
MAE: 170.5557617801548
MSE: 63013.68075778485
R^2: 0.36233044154224947


Observamos que tenemos mejores resultados en las predicciones de los valores de energía baja en comparación con las predicciones de los valores de energía alta.

### Segunda Parte: Cambiar a un modelo de clasificación
Para el problema de clasificación elegimos el DecisionTreeClassifier. Nos interesa utilizar su hiper-parámetro "weight_class" para poder lidiar con el desbalanceo de las clases.

In [2]:
# Importamos
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.preprocessing import StandardScaler

# Para simplificar nuestro set de datos, ahora con la variable de energía "alta" = 1 o "baja" = 0, borramos la variable "energy"
data_sotavento = data_sotavento.drop(columns=['energy'])

# Definimos nuestro data (atributos meteorolócios) y nuestro target (clase de energía)
X, y = data_sotavento.filter(regex=".13"), data_sotavento.energy_level

# Hacemos un split de los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=100472303)

# Hacemos nuestro pipeline
pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('Decision Tree', DecisionTreeClassifier(class_weight='balanced', random_state=100472303))
])

# Definimos los parámetros para la búsqueda en rejilla
param_grid = {
    'Decision Tree__criterion': ['gini', 'entropy'],
    'Decision Tree__max_depth': [None, 2, 3, 4, 5, 6],
    'Decision Tree__min_samples_split': [2, 5, 10],
    'Decision Tree__min_samples_leaf': [1, 2, 3, 4, 5]
}

# Creamos el objeto GridSearchCV
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='accuracy')

# Entrenamos el modelo
grid.fit(X_train, y_train)

# Imprimimos los mejores parámetros
print("Best parameters: ", grid.best_params_)

# Hacemos predicciones con el mejor modelo
y_pred = grid.predict(X_test)

# Calculamos métricas
accuracy = accuracy_score(y_test, y_pred)
precision_recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred)

# Imprimimos resultados
print(f"Accuracy: {accuracy}\nPrecision-Recall: {precision_recall}\nF1: {f1}\nROC AUC: {roc_auc}\n")

Best parameters:  {'Decision Tree__criterion': 'entropy', 'Decision Tree__max_depth': 2, 'Decision Tree__min_samples_leaf': 1, 'Decision Tree__min_samples_split': 2}
Accuracy: 0.8385449904275686
Precision-Recall: 0.7080103359173127
F1: 0.6841448189762797
ROC AUC: 0.7946831340603512



# 7. Conclusiones 

Inicialmente hemos decidido aplicar una búsqueda en rejilla para llevar a cabo la implementación del modelo haciendo uso de los hiperparámetros que mejor se adapten. Obteniendo como resultado:

    -Criterion = Entropy
    -Max_depth = 2
    -Min_samples_leaf = 1
    -Min_samples_split = 2

Gracias a estos hiper parámetros, y a que hacemos uso del balanceo de pesos, el cuál se encarga de otorgar relevancia equitativa a clases con menor frecuencia de aparición que las que tienen una mayor frecuencia, nuestro modelo consigue acertar las predicciones de los datos de eneregía con un 83% de presición lo cual es una tasa elevada. Adicional a ello, acierta con una exactitud bastante aceptable a la hora de definir casos que realmente son positivos (Presicion-Recall) casi en un 71% de las  veces. Para finalizar destacamos que nuestro modelo presenta valores de F1 Score de 0.684 siendo valores bastante sólidos y indicando un buen equilibrio entre precisión y recall, y una elevada capacidad de discriminar entre las clases positivas y negativas, debido a que obtiene un valor bastante cercano al 1 (0,79 puntos) de ROC AUC.

# 8. Implementación ChatGpt 

A lo largo de la práctica nos hemos apoyado en la utilización de ChatGPT a la hora de necesitar contenido con mayor detalle o algún ejemplo de cómo hacer un uso correcto de la librería de scikit-learn. Además, nos ha servido para la verificación de sintaxis de nuestro código y en el relleno del mismo, debido a que son unas de las funciones que ofrece la extensión de Copilot.